<a href="https://colab.research.google.com/github/ghazaletlbn/machinelearning/blob/main/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install shap
import shap
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
sales_train =  pd.read_csv('drive/My Drive/dataset/hw3/sales_train.csv', parse_dates = ['date'], infer_datetime_format = True, dayfirst = True)
item_categories = pd.read_csv('drive/My Drive/dataset/hw3/item_categories.csv')
items = pd.read_csv('drive/My Drive/dataset/hw3/items.csv')
shops = pd.read_csv('drive/My Drive/dataset/hw3/shops.csv')
test = pd.read_csv('drive/My Drive/dataset/hw3/test.csv')

In [29]:
sales_train.tail()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2935844,2015-10-10,33,25,7409,299.0,1.0
2935845,2015-10-09,33,25,7460,299.0,1.0
2935846,2015-10-14,33,25,7459,349.0,1.0
2935847,2015-10-22,33,25,7440,299.0,1.0
2935848,2015-10-03,33,25,7460,299.0,1.0


In [30]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [31]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [32]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [33]:
sales_train['date_block_num'] =sales_train['date_block_num'].map(str)
data =sales_train.groupby(['date_block_num','item_id','shop_id']).sum().reset_index()
del data['item_price']
data.tail(20)


The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.


,date_block_num,item_id,shop_id,item_cnt_day
1609104,9,22167,18,1.0
1609105,9,22167,21,2.0
1609106,9,22167,22,58.0
1609107,9,22167,24,3.0
1609108,9,22167,25,9.0
1609109,9,22167,26,3.0
1609110,9,22167,27,9.0
1609111,9,22167,28,3.0
1609112,9,22167,29,4.0
1609113,9,22167,30,1.0


In [34]:
data = data.pivot_table(index=['item_id','shop_id'], columns = 'date_block_num', values = 'item_cnt_day', fill_value = 0).reset_index()
data.head(20)

date_block_num,item_id,shop_id,0,1,10,11,12,13,14,15,...,30,31,32,33,4,5,6,7,8,9
0,0,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,55,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,2,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
data.shape

(424124, 36)

In [36]:
test = pd.merge(test,data,how='left',on=['shop_id','item_id']).fillna(0)
test.head()

,ID,shop_id,item_id,0,1,10,11,12,13,14,...,30,31,32,33,4,5,6,7,8,9
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
x_test = test.drop(['ID','shop_id','item_id'],axis=1)
x_test

,0,1,10,11,12,13,14,15,16,17,...,30,31,32,33,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
214196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,1.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
data = data.iloc[:, 0:34].reset_index(drop=True)

data


date_block_num,item_id,shop_id,0,1,10,11,12,13,14,15,...,29,3,30,31,32,33,4,5,6,7
0,0,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,55,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,2,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,22168,12,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424120,22168,16,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424121,22168,42,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424122,22168,43,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
x_train = data.iloc[:340_000,:-1]
y_train = data.iloc[:340_000,-1:]
x_validation = data.iloc[340_000:,:-1]
y_validation = data.iloc[340_000:,-1:]
print('x_train:     ',x_train.shape)
print('y_train:     ',y_train.shape)
print('x_validation:',x_validation.shape)
print('y_validation:',y_validation.shape)
print('x_test:      ',x_test.shape)

x_train:      (340000, 33)
y_train:      (340000, 1)
x_validation: (84124, 33)
y_validation: (84124, 1)
x_test:       (214200, 34)


In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_validation_scaled = scaler.fit_transform(x_validation)
linear = LinearRegression()
linear.fit(x_train_scaled,y_train)
y_pred = linear.predict(x_validation_scaled)
print('score: ',linear.score(x_validation_scaled,y_validation))
print('RMSE: ',mean_squared_error(y_pred, y_validation, squared=False))

score:  0.29915332012898654
RMSE:  4.719529339993369


In [51]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
lasso = Lasso(alpha=0.01)
lasso.fit(x_train_scaled,y_train)
y_pred2 = lasso.predict(x_validation_scaled)
print('score: ',lasso.score(x_validation_scaled,y_validation))
print('RMSE: ',mean_squared_error(y_pred2, y_validation, squared=False))

score:  0.2978376750131283
RMSE:  4.723957066207265
